In [0]:
import numpy as np
import pandas as pd
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Embedding, Flatten
from keras.layers.embeddings import Embedding
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
import nltk
import re
from bs4 import BeautifulSoup
import gensim
from nltk.tokenize import word_tokenize
import string
from keras.initializers import Constant
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving IMDB Dataset.csv to IMDB Dataset.csv
User uploaded file "IMDB Dataset.csv" with length 66212309 bytes


In [0]:
df=pd.DataFrame()
df=pd.read_csv('IMDB Dataset.csv',encoding='utf-8') 

In [0]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [0]:
df['review']=df['review'].apply(denoise_text)
review_lines=list()
lines=df['review'].values.tolist()
for line in lines:
  tokens=word_tokenize(line)
  tokens=[word.lower() for word in tokens]
  #table=str.maketrans('','',string.punctuation)
  stripped=[re.sub(r'[^a-zA-Z#]','',w) for w in tokens]
  words=[w for w in stripped if w.isalpha()]
  stop_words=set(stopwords.words('english'))
  words=[w for w in words if w not in stop_words]
  review_lines.append(words)
  

/usr/local/lib/python2.7/dist-packages/bs4/builder/_htmlparser.py:78: UserWarning: EOF in middle of entity or char ref
  warnings.warn(msg)


In [0]:
model=gensim.models.Word2Vec(sentences=review_lines,size=100,window=5,workers=4,min_count=1)
words=list(model.wv.vocab)
print("Vocabulary Size of word to vec %d" % (len(words)))

Vocabulary Size of word to vec 180800


In [0]:

#save_model
filename="imdb_word2vec.txt"
model.wv.save_word2vec_format(filename,binary=False)

/usr/local/lib/python2.7/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import os
embeddings_index={}
f=open(os.path.join('',filename))
for itr in f:
  values=itr.split()
  word=values[0]
  coefs=np.asarray(values[1:])
  embeddings_index[word]=coefs
  
    
    
f.close()

In [0]:
def IMDB_sentiment_label_changer(label):
  if label=="positive":
    label=0
  elif (label=='negative'):
    label=1
  return label

In [0]:
#converting data to a 2d tensor
tokenize_obj=Tokenizer()
tokenize_obj.fit_on_texts(review_lines)
sequences=tokenize_obj.texts_to_sequences(review_lines)

max_length=100
review_pad=pad_sequences(sequences,maxlen=max_length)
df['sentiment']=df['sentiment'].apply(IMDB_sentiment_label_changer)
sentiment_pad=df['sentiment'].values

#print ("Shape of Review tensor %d" % (review_pad.shape))
#print ("Shape of Sentiment tensor %d" % (sentiment_pad.shape))

In [0]:
num_words=len(tokenize_obj.word_index)+1
embedding_matrix=np.zeros((num_words,100))

for  word, i in tokenize_obj.word_index.items():
  if i>num_words:
    continue
  embedding_vector=embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector

In [0]:
np.unique(embedding_matrix)

array([-6.105117 , -6.0891943, -5.8666587, ...,  5.3796687,  5.4950747,
        5.590891 ])

In [0]:
indices=np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad=review_pad[indices]
sentiment_pad=sentiment_pad[indices]
num_validation_steps=int(0.2*review_pad.shape[0])
X_train_pad=review_pad[:-num_validation_steps]
Y_train=sentiment_pad[:-num_validation_steps]
X_test_pad=review_pad[-num_validation_steps:]
Y_test=sentiment_pad[-num_validation_steps:]



In [0]:
EMBEDDING_DIM=100
print('Build model...')
vocab_size=len(tokenize_obj.word_index)+1
model=Sequential()

model.add(Embedding(vocab_size,EMBEDDING_DIM,embeddings_initializer=Constant(embedding_matrix),input_length=max_length,trainable=False))
model.add(GRU(units=32,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model

Build model...


NameError: ignored

In [0]:
model.fit(X_train_pad,Y_train,batch_size=128,epochs=25,validation_data=(X_test_pad,Y_test),verbose=2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/25
 - 61s - loss: 0.5074 - acc: 0.7437 - val_loss: 0.3410 - val_acc: 0.8544
Epoch 2/25
 - 55s - loss: 0.3780 - acc: 0.8355 - val_loss: 0.3184 - val_acc: 0.8631
Epoch 3/25
 - 55s - loss: 0.3495 - acc: 0.8498 - val_loss: 0.3024 - val_acc: 0.8709
Epoch 4/25
 - 54s - loss: 0.3331 - acc: 0.8570 - val_loss: 0.2980 - val_acc: 0.8731
Epoch 5/25
 - 54s - loss: 0.3242 - acc: 0.8614 - val_loss: 0.2971 - val_acc: 0.8711
Epoch 6/25
 - 55s - loss: 0.3198 - acc: 0.8643 - val_loss: 0.2895 - val_acc: 0.8752
Epoch 7/25
 - 54s - loss: 0.3112 - acc: 0.8669 - val_loss: 0.2876 - val_acc: 0.8787
Epoch 8/25
 - 54s - loss: 0.3070 - acc: 0.8686 - val_loss: 0.2870 - val_acc: 0.8794
Epoch 9/25
 - 54s - loss: 0.3003 - acc: 0.8721 - val_loss: 0.2837 - val_acc: 0.8808
Epoch 10/25
 - 52s - loss: 0.3004 - acc: 0.8716 - val_loss: 0.2874 - val_acc: 0.8771
Epoch 11/25
 - 53s - loss: 0.2952 - acc: 0.8746 - val_loss: 0.2887 - val_acc: 0.8772
Epoch 12/25
 - 55s - los

In [0]:
loss, accuracy=model.evaluate(X_test_pad,Y_test,batch_size=128)
print("Accuracy %d" %(100*accuracy))

10000/10000 [==============================] - 5s 535us/step
Accuracy 88


In [0]:
sample_1="Overall the movie was nice"
sample_2="It was a mix of good and bad movie"
sample=[sample_1,sample_2]

seq=tokenize_obj.texts_to_sequences(sample)
test=pad_sequences(seq,maxlen=100)

model.predict(test)

array([[0.12469129],
       [0.6673398 ]], dtype=float32)

In [0]:

.